In [118]:
import numpy as np
from scipy.sparse.linalg import LinearOperator, eigsh
import pandas as pd
import seaborn as sns

from cvx.covariance.ewma import _ewma_cov
from experiments.utils.low_rank_ewma import _ewma_low_rank, low_rank_iterated_ewma
from experiments.utils.experiment_utils import log_likelihood_low_rank, log_likelihood_regularized, realized_volas, realized_covariance
from cvx.covariance.regularization import *
from cvx.covariance.regularization import em_regularize_covariance

sns.set()
sns.set(font_scale=1.5)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [113]:
returns = pd.read_csv('../data/HF_Returns_Stocks.csv', index_col=0, parse_dates=True, encoding='latin1', skiprows=1)

nyse_tickers_temp = [
    'JPM', 'GS', 'KO', 'DIS', 'IBM', 'CAT', 'CVX', 'XOM', 'T', 'GE',
    'MRK', 'VZ', 'PFE', 'WMT', 'C', 'HD', 'BA', 'UNH', 'MMM', 'TRV',
    'MCD', 'NKE', 'AAP', 'V', 'JNJ', 'INTC', 'MSFT', 'AAPL', 'GOOGL', 'AMZN',
    'BRK.B', 'CSCO', 'TSLA', 'PG', 'ABT', 'VLO', 'RTX', 'HON', 'LMT', 'TXN',
    'COST', 'PEP', 'UNP', 'WFC', 'CVS', 'ORCL', 'XRX', 'TMO', 'NSC'
]

nyse_tickers = [ticker for ticker in nyse_tickers_temp if ticker in returns.columns]

returns = returns[["Time"] + nyse_tickers]

In [157]:
(returns.drop(["Time"], axis=1).loc[start_date].cov(ddof=0)*77).head()

,JPM,GS,KO,IBM,CAT,CVX,XOM,GE,MRK,VZ,...,TXN,COST,PEP,UNP,WFC,CVS,ORCL,XRX,TMO,NSC
JPM,0.000100,0.000039,0.000033,0.000030,0.000044,0.000023,0.000045,0.000048,-0.000109,0.000040,...,0.000070,0.000047,0.000026,0.000026,0.000053,0.000028,0.000052,0.000071,0.000008,0.000013
GS,0.000039,0.000126,0.000057,0.000037,0.000027,0.000036,0.000046,0.000060,0.000023,0.000045,...,0.000082,0.000048,0.000055,0.000031,0.000037,0.000034,0.000068,0.000048,0.000036,0.000030
KO,0.000033,0.000057,0.000108,0.000046,0.000038,0.000043,0.000038,0.000043,0.000099,0.000020,...,0.000053,0.000078,0.000035,0.000039,0.000020,0.000045,0.000063,0.000046,0.000018,0.000026
IBM,0.000030,0.000037,0.000046,0.000067,0.000040,0.000027,0.000026,0.000035,0.000027,0.000019,...,0.000048,0.000054,0.000042,0.000026,0.000022,0.000037,0.000056,0.000054,0.000017,0.000018
CAT,0.000044,0.000027,0.000038,0.000040,0.000123,0.000023,0.000043,0.000051,-0.000074,0.000062,...,0.000090,0.000046,-0.000020,0.000029,0.000040,0.000070,0.000035,0.000049,0.000041,0.000032


In [155]:
pd.DataFrame(cov, index=returns.drop(["Time"], axis=1).columns, columns=returns.drop(["Time"], axis=1).columns).head()

,JPM,GS,KO,IBM,CAT,CVX,XOM,GE,MRK,VZ,...,TXN,COST,PEP,UNP,WFC,CVS,ORCL,XRX,TMO,NSC
JPM,0.000100,0.000040,0.000034,0.000031,0.000044,0.000024,0.000046,0.000048,-0.000108,0.000040,...,0.000070,0.000048,0.000026,0.000026,0.000054,0.000029,0.000052,0.000071,0.000009,0.000013
GS,0.000040,0.000130,0.000060,0.000041,0.000029,0.000038,0.000049,0.000060,0.000032,0.000045,...,0.000085,0.000054,0.000058,0.000033,0.000039,0.000038,0.000070,0.000051,0.000038,0.000028
KO,0.000034,0.000060,0.000110,0.000048,0.000039,0.000044,0.000040,0.000043,0.000106,0.000019,...,0.000055,0.000082,0.000037,0.000040,0.000021,0.000048,0.000064,0.000048,0.000019,0.000025
IBM,0.000031,0.000041,0.000048,0.000070,0.000042,0.000028,0.000029,0.000035,0.000034,0.000019,...,0.000050,0.000059,0.000044,0.000027,0.000023,0.000040,0.000057,0.000056,0.000018,0.000016
CAT,0.000044,0.000029,0.000039,0.000042,0.000124,0.000024,0.000045,0.000051,-0.000069,0.000062,...,0.000092,0.000049,-0.000019,0.000030,0.000041,0.000072,0.000036,0.000051,0.000042,0.000031


In [153]:
cov = 0

for i in range(0, 77):
    r_temp = returns.drop(["Time"], axis=1).loc[start_date].iloc[i]
    cov += np.outer(r_temp, r_temp)

In [132]:
temp = dict(realized_covariance(returns.drop(["Time"], axis=1)))

In [131]:
returns.iloc[:100].drop(["Time"], axis=1).cov()

,JPM,GS,KO,IBM,CAT,CVX,XOM,GE,MRK,VZ,...,TXN,COST,PEP,UNP,WFC,CVS,ORCL,XRX,TMO,NSC
JPM,1.138708e-06,3.778599e-07,3.108312e-07,3.523876e-07,5.106852e-07,2.546576e-07,4.654844e-07,5.150470e-07,-1.164219e-06,4.382232e-07,...,6.427015e-07,5.116101e-07,2.873150e-07,2.677462e-07,5.883644e-07,2.940001e-07,6.542838e-07,9.220778e-07,1.218459e-07,1.926161e-07
GS,3.778599e-07,1.691664e-06,6.648189e-07,3.866074e-07,3.402241e-07,3.668424e-07,4.404406e-07,7.489543e-07,3.768763e-07,4.429272e-07,...,8.666070e-07,5.532188e-07,5.724941e-07,3.596692e-07,5.067114e-07,4.221323e-07,8.490828e-07,5.096690e-07,4.980377e-07,4.202166e-07
KO,3.108312e-07,6.648189e-07,1.258586e-06,4.848997e-07,4.529705e-07,4.384623e-07,3.575128e-07,4.846958e-07,1.028985e-06,2.418462e-07,...,6.093062e-07,8.197217e-07,4.089203e-07,4.121022e-07,2.152594e-07,5.945810e-07,6.667136e-07,4.117163e-07,2.119120e-07,2.692161e-07
IBM,3.523876e-07,3.866074e-07,4.848997e-07,9.111688e-07,3.990319e-07,2.888108e-07,3.608680e-07,3.952637e-07,2.357129e-07,2.857369e-07,...,6.696516e-07,3.994491e-07,4.904690e-07,1.941225e-07,3.021396e-07,3.001106e-07,7.339489e-07,7.563265e-07,2.798811e-07,2.113938e-07
CAT,5.106852e-07,3.402241e-07,4.529705e-07,3.990319e-07,1.721208e-06,3.613406e-07,2.974003e-07,6.264697e-07,-7.716816e-07,6.540908e-07,...,8.156537e-07,5.894804e-07,-1.601180e-07,3.696127e-07,4.005516e-07,7.539797e-07,7.485874e-07,9.951477e-07,5.104738e-07,2.802411e-07
CVX,2.546576e-07,3.668424e-07,4.384623e-07,2.888108e-07,3.613406e-07,7.677586e-07,4.192626e-07,3.731062e-07,7.774518e-07,2.153876e-07,...,5.345508e-07,6.524292e-07,2.845257e-07,3.254018e-07,1.687933e-07,4.424334e-07,6.232619e-07,4.402969e-07,3.248578e-07,1.069292e-07
XOM,4.654844e-07,4.404406e-07,3.575128e-07,3.608680e-07,2.974003e-07,4.192626e-07,1.081539e-06,4.494012e-07,-6.473110e-07,5.790847e-07,...,9.096893e-07,5.951402e-07,3.661402e-07,2.033725e-07,4.599608e-07,4.138645e-07,5.373939e-07,4.954131e-07,4.573779e-07,2.013049e-07
GE,5.150470e-07,7.489543e-07,4.846958e-07,3.952637e-07,6.264697e-07,3.731062e-07,4.494012e-07,1.154483e-06,-8.792707e-07,7.242118e-07,...,1.078873e-06,5.895916e-07,2.892338e-07,3.244117e-07,5.828127e-07,8.085358e-07,7.512074e-07,8.040739e-07,5.742148e-07,3.485409e-07
MRK,-1.164219e-06,3.768763e-07,1.028985e-06,2.357129e-07,-7.716816e-07,7.774518e-07,-6.473110e-07,-8.792707e-07,1.784794e-05,-3.105215e-06,...,-1.723936e-06,1.214234e-06,8.960425e-07,5.850370e-07,-1.736764e-06,-8.356635e-07,9.888397e-07,-1.760956e-06,1.834556e-08,-3.253659e-07
VZ,4.382232e-07,4.429272e-07,2.418462e-07,2.857369e-07,6.540908e-07,2.153876e-07,5.790847e-07,7.242118e-07,-3.105215e-06,1.909790e-06,...,1.248545e-06,4.375924e-07,3.593859e-07,1.295660e-07,6.993092e-07,7.575203e-07,2.783031e-07,6.812816e-07,4.772988e-07,3.812813e-07


In [133]:
temp[start_date]


,JPM,GS,KO,IBM,CAT,CVX,XOM,GE,MRK,VZ,...,TXN,COST,PEP,UNP,WFC,CVS,ORCL,XRX,TMO,NSC
JPM,1.292456e-06,5.066196e-07,4.322078e-07,3.906127e-07,5.718164e-07,3.046120e-07,5.890521e-07,6.243866e-07,-1.419483e-06,5.198425e-07,...,9.079388e-07,6.130398e-07,3.349293e-07,3.325087e-07,6.917243e-07,3.667200e-07,6.732380e-07,9.209754e-07,1.066507e-07,1.718445e-07
GS,5.066196e-07,1.633368e-06,7.340613e-07,4.860697e-07,3.511039e-07,4.644294e-07,5.935386e-07,7.754759e-07,2.924430e-07,5.865549e-07,...,1.067024e-06,6.260196e-07,7.152290e-07,4.072719e-07,4.824225e-07,4.356332e-07,8.843361e-07,6.210078e-07,4.624711e-07,3.904390e-07
KO,4.322078e-07,7.340613e-07,1.397136e-06,5.948970e-07,4.926929e-07,5.540767e-07,4.987909e-07,5.548008e-07,1.289320e-06,2.558754e-07,...,6.887271e-07,1.018249e-06,4.585376e-07,5.027908e-07,2.565728e-07,5.879341e-07,8.183169e-07,5.959994e-07,2.342837e-07,3.360018e-07
IBM,3.906127e-07,4.860697e-07,5.948970e-07,8.759327e-07,5.215575e-07,3.454876e-07,3.407130e-07,4.506893e-07,3.534967e-07,2.506168e-07,...,6.251859e-07,7.062666e-07,5.396010e-07,3.418657e-07,2.878059e-07,4.822444e-07,7.217821e-07,7.004587e-07,2.170996e-07,2.290842e-07
CAT,5.718164e-07,3.511039e-07,4.926929e-07,5.215575e-07,1.601267e-06,2.976008e-07,5.591630e-07,6.666407e-07,-9.554404e-07,8.049686e-07,...,1.172209e-06,5.945655e-07,-2.602594e-07,3.796619e-07,5.208333e-07,9.050520e-07,4.548936e-07,6.421360e-07,5.271847e-07,4.171140e-07
CVX,3.046120e-07,4.644294e-07,5.540767e-07,3.454876e-07,2.976008e-07,7.791953e-07,5.027383e-07,4.195009e-07,1.038607e-06,1.529011e-07,...,5.948017e-07,7.687975e-07,3.608500e-07,4.323305e-07,2.389138e-07,5.606639e-07,5.727775e-07,2.936402e-07,3.393444e-07,1.792894e-07
XOM,5.890521e-07,5.935386e-07,4.987909e-07,3.407130e-07,5.591630e-07,5.027383e-07,1.056322e-06,5.701419e-07,-8.111538e-07,6.791985e-07,...,9.552998e-07,9.189280e-07,3.844648e-07,3.963199e-07,5.412437e-07,7.414491e-07,6.239279e-07,6.221125e-07,5.514139e-07,2.349554e-07
GE,6.243866e-07,7.754759e-07,5.548008e-07,4.506893e-07,6.666407e-07,4.195009e-07,5.701419e-07,1.231382e-06,-1.279813e-06,9.105377e-07,...,1.291473e-06,6.987272e-07,3.007931e-07,4.251387e-07,6.744749e-07,9.591939e-07,6.749085e-07,7.733185e-07,5.912417e-07,3.820593e-07
MRK,-1.419483e-06,2.924430e-07,1.289320e-06,3.534967e-07,-9.554404e-07,1.038607e-06,-8.111538e-07,-1.279813e-06,2.241020e-05,-3.943463e-06,...,-2.269252e-06,1.430357e-06,1.079834e-06,7.091364e-07,-2.264525e-06,-1.278266e-06,1.219619e-06,-2.097828e-06,-8.999442e-08,-3.792650e-07
VZ,5.198425e-07,5.865549e-07,2.558754e-07,2.506168e-07,8.049686e-07,1.529011e-07,6.791985e-07,9.105377e-07,-3.943463e-06,2.037160e-06,...,1.412379e-06,5.828295e-07,4.400723e-07,1.671688e-07,8.170452e-07,1.002366e-06,1.043403e-07,6.125421e-07,6.406306e-07,4.524239e-07


In [128]:
temp[start_date]


,JPM,GS,KO,IBM,CAT,CVX,XOM,GE,MRK,VZ,...,TXN,COST,PEP,UNP,WFC,CVS,ORCL,XRX,TMO,NSC
JPM,9.951908e-05,0.000039,0.000033,0.000030,0.000044,0.000023,0.000045,0.000048,-1.093002e-04,0.000040,...,0.000070,0.000047,0.000026,0.000026,0.000053,0.000028,0.000052,0.000071,0.000008,0.000013
GS,3.900971e-05,0.000126,0.000057,0.000037,0.000027,0.000036,0.000046,0.000060,2.251811e-05,0.000045,...,0.000082,0.000048,0.000055,0.000031,0.000037,0.000034,0.000068,0.000048,0.000036,0.000030
KO,3.328000e-05,0.000057,0.000108,0.000046,0.000038,0.000043,0.000038,0.000043,9.927762e-05,0.000020,...,0.000053,0.000078,0.000035,0.000039,0.000020,0.000045,0.000063,0.000046,0.000018,0.000026
IBM,3.007717e-05,0.000037,0.000046,0.000067,0.000040,0.000027,0.000026,0.000035,2.721925e-05,0.000019,...,0.000048,0.000054,0.000042,0.000026,0.000022,0.000037,0.000056,0.000054,0.000017,0.000018
CAT,4.402986e-05,0.000027,0.000038,0.000040,0.000123,0.000023,0.000043,0.000051,-7.356891e-05,0.000062,...,0.000090,0.000046,-0.000020,0.000029,0.000040,0.000070,0.000035,0.000049,0.000041,0.000032
CVX,2.345512e-05,0.000036,0.000043,0.000027,0.000023,0.000060,0.000039,0.000032,7.997272e-05,0.000012,...,0.000046,0.000059,0.000028,0.000033,0.000018,0.000043,0.000044,0.000023,0.000026,0.000014
XOM,4.535701e-05,0.000046,0.000038,0.000026,0.000043,0.000039,0.000081,0.000044,-6.245884e-05,0.000052,...,0.000074,0.000071,0.000030,0.000031,0.000042,0.000057,0.000048,0.000048,0.000042,0.000018
GE,4.807777e-05,0.000060,0.000043,0.000035,0.000051,0.000032,0.000044,0.000095,-9.854562e-05,0.000070,...,0.000099,0.000054,0.000023,0.000033,0.000052,0.000074,0.000052,0.000060,0.000046,0.000029
MRK,-1.093002e-04,0.000023,0.000099,0.000027,-0.000074,0.000080,-0.000062,-0.000099,1.725586e-03,-0.000304,...,-0.000175,0.000110,0.000083,0.000055,-0.000174,-0.000098,0.000094,-0.000162,-0.000007,-0.000029
VZ,4.002787e-05,0.000045,0.000020,0.000019,0.000062,0.000012,0.000052,0.000070,-3.036467e-04,0.000157,...,0.000109,0.000045,0.000034,0.000013,0.000063,0.000077,0.000008,0.000047,0.000049,0.000035


In [2]:
import refinitiv.data as rd
from refinitiv.data.content import historical_pricing
from refinitiv.data.content.historical_pricing import Intervals
from refinitiv.data.content.historical_pricing import Adjustments

In [16]:
def get_data(stock_symbol, start_date, end_date):
    response = historical_pricing.summaries.Definition(
            stock_symbol,
            start=start_date,
            end=end_date,
            interval=Intervals.HOURLY,
        ).get_data()
    
    return response.data.df
    

In [4]:
rd.open_session()

import datetime
from tqdm import tqdm

nasdaq_stocks = [
    "AAPL.O", "MSFT.O",
      "AMZN.O", "GOOGL.O", "FB.O",
    "TSLA.O", "NVDA.O", "ADBE.O", "NFLX.O", "INTC.O",
    "CSCO.O", "AMGN.O", "PYPL.O", "AMAT.O", "AVGO.O",
    # Add more NASDAQ-listed stocks here
]


# Define the date range for historical data
end_date = datetime.date.today()
start_date = end_date - datetime.timedelta(days=365*15)  # Adjust the number of days as needed

# Split start_date and end_date into 15 separate years to get data one year at a
# time

start_dates = [
    datetime.date(start_date.year + i, 1, 1) for i in range(15)
]
end_dates = [
    datetime.date(start_date.year + i, 12, 31) for i in range(15)
]

In [6]:
start_dates[0]

datetime.date(2008, 1, 1)

In [7]:
end_dates[0]

datetime.date(2008, 12, 31)

In [ ]:
nyse_stocks = [
    "T", "JPM", "VZ", "WMT", "KO",
    "BAC", "PFE", "CVX", "MRK", "XOM",
    "DIS", "BA", "IBM", "MCD", "HD",
    # Add more NYSE-listed stocks here
]


In [22]:
get_data("MCD", start_dates[13], end_dates[13])

""


In [45]:
rd.open_session()

import datetime
from tqdm import tqdm

nasdaq_stocks = [
    "AAPL.O", "MSFT.O",
      "AMZN.O", "GOOGL.O", "FB.O",
    "TSLA.O", "NVDA.O", "ADBE.O", "NFLX.O", "INTC.O",
    "CSCO.O", "AMGN.O", "PYPL.O", "AMAT.O", "AVGO.O",
    # Add more NASDAQ-listed stocks here
]


# Define the date range for historical data
end_date = datetime.date.today()
start_date = end_date - datetime.timedelta(days=365*15)  # Adjust the number of days as needed

# Split start_date and end_date into 15 separate years to get data one year at a
# time

start_dates = [
    datetime.date(start_date.year + i, 1, 1) for i in range(15)
]
end_dates = [
    datetime.date(start_date.year + i, 12, 31) for i in range(15)
]


# Define the interval (30 minutes)
interval = datetime.timedelta(minutes=30)

# Create an empty DataFrame to store the data
historical_data = {}

# Loop through the list of stocks and fetch data
for stock_symbol in tqdm(nasdaq_stocks):
    try:
        # response = historical_pricing.summaries.Definition(
        #     stock_symbol,
        #     start=start_date,
        #     end=end_date,
        #     interval=Intervals.THIRTY_MINUTES,
        # ).get_data()
    
        for i in range(len(start_dates)):
            data_temp = get_data(stock_symbol, start_dates[i], end_dates[i])
            if i == 0:
                data = data_temp
            else:
                data = pd.concat([data, data_temp], axis=0)
    except:
        print(f"Failed to fetch data for {stock_symbol}")
        continue
    historical_data[stock_symbol] = data


rd.close_session()

  7%|▋         | 1/15 [00:11<02:35, 11.09s/it]

Failed to fetch data for AAPL.O


 13%|█▎        | 2/15 [00:22<02:26, 11.26s/it]

In [39]:
start_date = end_date - datetime.timedelta(days=365)

In [36]:
rd.open_session()


<refinitiv.data.session.Definition object at 0x7fecd88d6ac0 {name='workspace'}>

In [40]:
response = historical_pricing.summaries.Definition(
            stock_symbol,
            start=start_date,
            end=end_date,
            interval=Intervals.THIRTY_MINUTES,
        ).get_data()

In [42]:
start_date

datetime.date(2022, 9, 21)

In [43]:
end_date

datetime.date(2023, 9, 21)

In [41]:
response.data.df

AVGO.O,HIGH_1,LOW_1,OPEN_PRC,TRDPRC_1,NUM_MOVES,ACVOL_UNS,HIGH_YLD,LOW_YLD,OPEN_YLD,YIELD,...,BID_NUMMOV,ASK_HIGH_1,ASK_LOW_1,OPEN_ASK,ASK,ASK_NUMMOV,MID_HIGH,MID_LOW,MID_OPEN,MID_PRICE
Timestamp,,,,,,,,,,,,,,,,,,,,,
2022-09-21 04:30:00,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,3.3541,3.3541,3.3541,3.3541,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2022-09-21 08:00:00,492.2,488.96,492.0,491.01,17,78,<NA>,<NA>,<NA>,<NA>,...,30,614.7,492.35,614.7,492.5,30,<NA>,<NA>,<NA>,<NA>
2022-09-21 08:30:00,491.3,490.98,491.0,491.23,5,14,<NA>,<NA>,<NA>,<NA>,...,4,492.5,492.5,492.5,492.5,4,<NA>,<NA>,<NA>,<NA>
2022-09-21 09:00:00,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,1,492.5,492.5,492.5,492.5,1,<NA>,<NA>,<NA>,<NA>
2022-09-21 09:30:00,491.23,491.2,491.2,491.23,2,3,<NA>,<NA>,<NA>,<NA>,...,4,500.0,492.5,492.5,500.0,4,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-20 21:30:00,829.98,829.0,829.5,829.94,45,220,<NA>,<NA>,<NA>,<NA>,...,10,830.3,829.75,829.75,830.3,10,<NA>,<NA>,<NA>,<NA>
2023-09-20 22:00:00,830.5,828.7,829.94,829.6,39,208,<NA>,<NA>,<NA>,<NA>,...,23,830.5,830.5,830.5,830.5,23,<NA>,<NA>,<NA>,<NA>
2023-09-20 22:30:00,829.75,828.71,829.6,828.98,49,253,<NA>,<NA>,<NA>,<NA>,...,17,830.0,829.6,830.0,829.75,17,<NA>,<NA>,<NA>,<NA>


In [33]:
response.data.df

AVGO.O,HIGH_1,LOW_1,OPEN_PRC,TRDPRC_1,NUM_MOVES,ACVOL_UNS,HIGH_YLD,LOW_YLD,OPEN_YLD,YIELD,...,BID_NUMMOV,ASK_HIGH_1,ASK_LOW_1,OPEN_ASK,ASK,ASK_NUMMOV,MID_HIGH,MID_LOW,MID_OPEN,MID_PRICE
Timestamp,,,,,,,,,,,,,,,,,,,,,
2022-09-21 04:30:00,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,3.3541,3.3541,3.3541,3.3541,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2022-09-21 08:00:00,492.2,488.96,492.0,491.01,17,78,<NA>,<NA>,<NA>,<NA>,...,30,614.7,492.35,614.7,492.5,30,<NA>,<NA>,<NA>,<NA>
2022-09-21 08:30:00,491.3,490.98,491.0,491.23,5,14,<NA>,<NA>,<NA>,<NA>,...,4,492.5,492.5,492.5,492.5,4,<NA>,<NA>,<NA>,<NA>
2022-09-21 09:00:00,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,1,492.5,492.5,492.5,492.5,1,<NA>,<NA>,<NA>,<NA>
2022-09-21 09:30:00,491.23,491.2,491.2,491.23,2,3,<NA>,<NA>,<NA>,<NA>,...,4,500.0,492.5,492.5,500.0,4,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-20 21:30:00,829.98,829.0,829.5,829.94,45,220,<NA>,<NA>,<NA>,<NA>,...,10,830.3,829.75,829.75,830.3,10,<NA>,<NA>,<NA>,<NA>
2023-09-20 22:00:00,830.5,828.7,829.94,829.6,39,208,<NA>,<NA>,<NA>,<NA>,...,23,830.5,830.5,830.5,830.5,23,<NA>,<NA>,<NA>,<NA>
2023-09-20 22:30:00,829.75,828.71,829.6,828.98,49,253,<NA>,<NA>,<NA>,<NA>,...,17,830.0,829.6,830.0,829.75,17,<NA>,<NA>,<NA>,<NA>


In [24]:
4+8

12

In [30]:
historical_data["AAPL.O"]

AAPL.O,HIGH_1,LOW_1,OPEN_PRC,TRDPRC_1,NUM_MOVES,ACVOL_UNS,HIGH_YLD,LOW_YLD,OPEN_YLD,YIELD,...,BID_NUMMOV,ASK_HIGH_1,ASK_LOW_1,OPEN_ASK,ASK,ASK_NUMMOV,MID_HIGH,MID_LOW,MID_OPEN,MID_PRICE
Timestamp,,,,,,,,,,,,,,,,,,,,,
2022-09-21 04:30:00,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.5864,0.5864,0.5864,0.5864,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2022-09-21 08:00:00,157.39,156.0,156.11,156.65,621,27876,<NA>,<NA>,<NA>,<NA>,...,907,168.7,0.0,0.0,156.65,907,<NA>,<NA>,<NA>,<NA>
2022-09-21 08:30:00,156.78,156.5,156.63,156.62,236,8850,<NA>,<NA>,<NA>,<NA>,...,453,156.95,156.54,156.72,156.7,453,<NA>,<NA>,<NA>,<NA>
2022-09-21 09:00:00,156.63,156.3,156.63,156.48,300,8871,<NA>,<NA>,<NA>,<NA>,...,362,156.7,156.36,156.69,156.49,362,<NA>,<NA>,<NA>,<NA>
2022-09-21 09:30:00,156.97,156.42,156.46,156.74,430,21885,<NA>,<NA>,<NA>,<NA>,...,400,157.0,156.46,156.46,156.8,400,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-20 21:30:00,177.55,175.45,175.5,175.6,1053,1715743,<NA>,<NA>,<NA>,<NA>,...,298,175.63,175.49,175.56,175.61,298,<NA>,<NA>,<NA>,<NA>
2023-09-20 22:00:00,176.5887,175.36,175.59,175.395,850,23969,<NA>,<NA>,<NA>,<NA>,...,266,175.65,175.4,175.61,175.43,266,<NA>,<NA>,<NA>,<NA>
2023-09-20 22:30:00,175.52,175.31,175.43,175.42,743,19427,<NA>,<NA>,<NA>,<NA>,...,170,175.52,175.38,175.42,175.42,170,<NA>,<NA>,<NA>,<NA>


In [7]:
response.data.df

GBP=,BID_HIGH_1,BID_LOW_1,OPEN_BID,BID,BID_NUMMOV,ASK_HIGH_1,ASK_LOW_1,OPEN_ASK,ASK,ASK_NUMMOV,MID_HIGH,MID_LOW,MID_OPEN,MID_PRICE
Timestamp,,,,,,,,,,,,,,
2023-09-21 15:00:00,1.2304,1.2283,1.2286,1.2294,1959,1.2308,1.2286,1.229,1.2299,1959,1.23055,1.2285,1.2288,1.22965
2023-09-21 15:30:00,1.23,1.2287,1.2294,1.2295,1762,1.2304,1.2291,1.2298,1.23,1762,1.23015,1.2289,1.2296,1.22975
2023-09-21 16:00:00,1.2299,1.2287,1.2295,1.2296,1585,1.2303,1.2291,1.2299,1.2301,1585,1.23005,1.229,1.2297,1.22985
2023-09-21 16:30:00,1.2306,1.2294,1.2299,1.2296,1429,1.231,1.2297,1.23,1.2297,1429,1.23075,1.22965,1.22995,1.22965
2023-09-21 17:00:00,1.2307,1.2293,1.2295,1.2303,1637,1.231,1.2296,1.2299,1.2307,1637,1.23075,1.2295,1.2297,1.2305
2023-09-21 17:30:00,1.2307,1.2296,1.2303,1.2297,1542,1.2311,1.2299,1.2307,1.2301,1542,1.23085,1.2298,1.2305,1.2299
2023-09-21 18:00:00,1.2304,1.2294,1.2298,1.2298,1349,1.2309,1.2297,1.23,1.2302,1349,1.23065,1.22965,1.2299,1.23
2023-09-21 18:30:00,1.2299,1.2288,1.2298,1.2288,1278,1.2302,1.229,1.2302,1.2292,1278,1.23,1.22895,1.23,1.229
2023-09-21 19:00:00,1.2296,1.2287,1.2287,1.2294,1207,1.2299,1.2289,1.2291,1.2295,1207,1.22965,1.22885,1.2289,1.22945


In [25]:
import refinitiv.data.eikon as ek
import datetime


In [35]:
ek.get_timeseries(['APPL.L'], 
                  start_date=datetime.timedelta(-750), 
                  end_date=datetime.timedelta(0), 
                  interval='hour')

Error with APPL.L: Invalid RIC
APPL.L: Invalid RIC | 


RDError: Error code Error | APPL.L: Invalid RIC | 

In [36]:
ek.get_timeseries(['AAPL.O'], 
                  start_date=datetime.timedelta(-365), 
                  end_date=datetime.timedelta(0), 
                  interval='hour')

AAPL.O,HIGH,LOW,OPEN,CLOSE,COUNT,VOLUME
Date,,,,,,
2022-09-22 09:00:00,154.96,153.02,153.02,154.55,2111,110976
2022-09-22 10:00:00,154.54,153.55,154.45,153.9,1063,53460
2022-09-22 11:00:00,154.06,153.55,153.9,153.96,657,18161
2022-09-22 12:00:00,154.0,152.92,153.95,153.53,2871,132585
2022-09-22 13:00:00,154.9288,152.36,154.0,153.94,6947,412603
...,...,...,...,...,...,...
2023-09-21 20:00:00,174.81,173.86,174.66,173.93,105068,10201077
2023-09-21 21:00:00,174.61,173.79,173.92,173.88,3557,10087007
2023-09-21 22:00:00,173.97,163.093667,173.895,173.85,2097,61580


In [38]:
ek.get_timeseries(['AAPL.O'], 
                  start_date=datetime.timedelta(-10000), 
                  end_date=datetime.timedelta(0), 
                  interval='hour')

AAPL.O,HIGH,LOW,OPEN,CLOSE,COUNT,VOLUME
Date,,,,,,
2022-09-21 09:00:00,157.39,156.0,156.11,156.62,857,36726
2022-09-21 10:00:00,156.97,156.3,156.63,156.74,730,30756
2022-09-21 11:00:00,156.8,156.41,156.78,156.65,543,22046
2022-09-21 12:00:00,156.88,156.36,156.68,156.82,1829,130277
2022-09-21 13:00:00,157.47,156.11,156.67,157.32,5709,398531
...,...,...,...,...,...,...
2023-09-21 20:00:00,174.81,173.86,174.66,173.93,105068,10201077
2023-09-21 21:00:00,174.61,173.79,173.92,173.88,3557,10087007
2023-09-21 22:00:00,173.97,163.093667,173.895,173.85,2097,61580


# intraday data with yfinance

In [39]:
import yfinance as yf

In [57]:
ticker = yf.Ticker("MSFT")
period = "1y"
interval = "2h"

start = "2020-01-01"
end = "2023-01-01"

end = datetime.datetime.today()

ticker.history(start=start, end=end, interval=interval)

# ticker.history(period=period, interval=interval)

MSFT: Invalid input - interval=2h is not supported. Valid intervals: , 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo]


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
